In [ ]:
!git clone https://github.com/BIOL359A-FoundationsOfQBio-Spr22/week6_sparseregression
!mkdir ./data
!cp week5_modelselection/data/* ./data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import sklearn as sk
import matplotlib.pyplot as plt
import ipywidgets as widgets

from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_validate

%matplotlib inline

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

TITLE_FONT = 20
LABEL_FONT = 16
TICK_FONT = 16
FIG_SIZE = (5,5)
COLORS= ["#008080","#CA562C"]

sns.set(font_scale=1.5, rc={'figure.figsize':FIG_SIZE}) 
sns.set_style("whitegrid",  {'axes.linewidth': 2, 'axes.edgecolor':'black'})
plt.rc("axes.spines", top=False, right=False)


In [2]:
dataset = pd.read_csv("data/data.csv")
def get_numerical(df):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_df = df.select_dtypes(include=numerics)
    return num_df
    
num_dataset = get_numerical(dataset)
num_dataset

,Start,End,G,U,bi,uni,duplex,Pos1,Pos2,Pos6,...,Pos18,Dif_5-3,Content+,Content-,Cons+,Cons-,Cons_Sum,Hyb19,target,Activity
0,195,213,3,9,-7.0,-1.9,-28.1,-1.1,-0.9,-3.3,...,-2.4,-1.3,2,6,2,6,4,0.001175,-8.1,24.7
1,197,215,2,9,-0.7,0.0,-24.2,-0.9,-0.9,-1.1,...,-2.1,-1.2,1,6,1,5,4,0.001175,-8.1,9.8
2,199,217,1,8,-1.5,0.0,-24.2,-2.1,-0.9,-0.9,...,-0.9,1.2,2,5,3,2,-1,0.001175,-11.4,50.5
3,201,219,1,6,-0.6,0.0,-26.7,-2.1,-2.2,-0.9,...,-0.9,1.2,3,3,3,3,0,0.001175,-14.5,83.7
4,203,221,2,4,-0.1,0.0,-30.3,-3.4,-3.3,-0.9,...,-2.1,1.3,4,2,2,3,1,0.001175,-17.8,60.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,717,735,6,4,-9.0,-0.3,-37.8,-2.2,-0.9,-3.4,...,-3.3,-1.1,5,3,8,6,-2,0.000789,-15.1,26.0
649,837,855,2,5,-2.8,0.0,-38.6,-2.4,-3.4,-3.4,...,-2.4,0.0,4,4,5,3,-2,0.000789,-24.8,48.0
650,972,990,7,4,-7.9,-2.3,-34.1,-2.4,-2.1,-2.1,...,-2.4,0.0,3,2,5,4,-1,0.000789,-5.2,36.0
651,1008,1026,5,6,-9.5,-0.3,-31.7,-2.2,-0.9,-2.1,...,-2.4,-0.2,3,3,2,4,2,0.000789,-11.6,25.0


In [3]:
def cross_validation(model, X, y, k=10):
    scores = cross_validate(model, X, y, cv=k,
                            scoring=('r2', 'neg_root_mean_squared_error'),
                            return_train_score=True)
    
    temp_model = model.fit(X, y)
    non_zero = np.count_nonzero(temp_model.coef_)
    return np.mean(scores['train_r2']),np.mean(scores['test_r2']), non_zero


def parity_plot(true, pred, r_squared=None, title='', alpha=None, color=None, hue=None):
    """
    plot true vs the predicted data
    inputs: 2 list-like (arrays) data structures
    """
    fig, ax = plt.subplots(1,1,figsize=(10, 8))
    if hue is not None:
        sns.scatterplot(x=true, y=pred, hue=hue)
    else: 
        if color is None: sns.scatterplot(x=true, y=pred)
        else: sns.scatterplot(x=true, y=pred, alpha=alpha, color=color)
    min_value = min(min(true), min(pred))
    max_value = max(max(true), max(pred))
    plt.plot([min_value, max_value],[min_value, max_value], '--', label="parity")
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    sns.despine()
    plt.text(1.01, 0.98, r"$R^2 = {0:.2f}$".format(r_squared),
         ha='left', va='top', size =LABEL_FONT,
         transform=ax.transAxes)
    plt.title('Parity Plot: {}'.format(title), size=TITLE_FONT)
    plt.legend(loc='best')
    plt.show()    
    
    

In [ ]:


def model_selection(X,y, degrees, interactions):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    
    poly = PolynomialFeatures(degrees, include_bias=True)
    # print(poly.get_feature_names_out(input_features=X_train.columns))
    X_train = poly.fit_transform(X_train)
    poly_features = poly.get_feature_names_out()
    X_test = poly.fit_transform(X_test, poly_features)
    X_train_df = pd.DataFrame(X_train, columns=poly_features)
    X_test_df = pd.DataFrame(X_test, columns=poly_features)
    interaction_list = [feat for feat in poly_features if len(feat.split())!=1]
    if not interactions:
        X_train_df = X_train_df.drop(interaction_list, axis=1)
        X_test_df = X_test_df.drop(interaction_list, axis=1)    
    
    
    X_scaler = StandardScaler().fit(X_train_df)
    y_scaler = StandardScaler().fit(y_train.values.reshape(-1,1))
    X_z = X_scaler.transform(X_train_df)
    y_z = y_scaler.transform(y_train.values.reshape(-1,1))
    
    alpha_dict = {}
    p_dict = {}
    i = 0
    lim = 64
    for alpha in [0, 1e-5, 1e-3, 1e-2, 1e-1, 0.5, 1, 5]:
        alpha_dict[str(alpha)] = {}
        p_dict[str(alpha)] = {}
        for ratio in [0, .1, .5, .7, .9, .95, .99, 1]:
            i += 1
            print(f"fitting and CV on {i} of {lim}")
            model = linear_model.ElasticNet(alpha=alpha, l1_ratio=ratio) 

            train_r2, test_r2, non_zero = cross_validation(model, X_z, y_z)
            alpha_dict[str(alpha)][str(ratio)] = f"{train_r2:.4f}/{test_r2:.4f}"
            p_dict[str(alpha)][str(ratio)] = non_zero
            
    cv_df = pd.DataFrame(alpha_dict)
    parameter_df = pd.DataFrame(p_dict)
    cv_df.index.name = "L1 Ratio"
    cv_df.style.set_caption("Alpha")
    parameter_df.index.name = "L1 Ratio"
    parameter_df.style.set_caption("Alpha")    
    return cv_df, parameter_df, X_train_df, X_test_df, y_train, y_test

def run():
    features = ['Start', 'End', 'G', 'U', 'bi', 'uni', 'duplex', 'Pos1', 'Pos2', 'Pos6',
       'Pos13', 'Pos14', 'Pos18', 'Dif_5-3', 'Content+', 'Content-', 'Cons+',
       'Cons-', 'Cons_Sum', 'Hyb19', 'target']
    X = num_dataset[features]
    y = num_dataset["Activity"]
    cv_df, parameter_df, X_train, X_test, y_train, y_test = model_selection(X,y,3,True)
    return cv_df, parameter_df, X_train, X_test, y_train, y_test 

cv_df, parameter_df, X_train, X_test, y_train, y_test = run()

print("Columns = alpha")
print("Rows = L1_ratio")
print("R^2 from TRAIN/VALIDATION:")
cv_df

fitting and CV on 1 of 64
fitting and CV on 2 of 64
fitting and CV on 3 of 64
fitting and CV on 4 of 64
fitting and CV on 5 of 64
fitting and CV on 6 of 64
fitting and CV on 7 of 64
fitting and CV on 8 of 64
fitting and CV on 9 of 64
fitting and CV on 10 of 64
fitting and CV on 11 of 64
fitting and CV on 12 of 64


In [ ]:
print("Number of parameters in each model:")
parameter_df

In [6]:
def test_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    parity_plot(y_train, y_train_pred.flatten(), r_squared =model.score(X_train, y_train), title="Training Data", color="grey", alpha=0.5)
    parity_plot(y_test, y_test_pred.flatten(), r_squared =model.score(X_test, y_test), title="Test Data", color="blue", alpha=1)


@widgets.interact_manual(alpha=[0, 1e-5, 1e-3, 1e-2, 1e-1, 0.5, 1, 5], l1_ratio=[.1, .5, .7, .9, .95, .99, 1])
def build_and_test(alpha, l1_ratio):
    model = linear_model.ElasticNet(alpha=alpha, l1_ratio=l1_ratio) 
    test_model(model, X_train, X_test, y_train, y_test)

interactive(children=(Dropdown(description='alpha', options=(0, 1e-05, 0.001, 0.01, 0.1, 0.5, 1, 5), value=0),…